In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import get_data
import statsmodels
from simulate import simulatedata

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
syms = ['BTCUSDT']
freq = '10min'
start_date = '2020-01-01'
end_date = '2020-01-03'

In [ ]:
raw_data = get_data.get_data(syms, freq, start_date, end_date)
raw_data

In [ ]:
dt = pd.DataFrame()
dt['close'] = raw_data['BTCUSDT']['close']
dt['ret'] = raw_data['BTCUSDT']['close'].pct_change()
dt

In [ ]:
order, seasonal_order = simulatedata.get_order(dt['ret'][1:])
order, seasonal_order

In [ ]:
if sum(seasonal_order) == 1: seasonal_order = (0, 0, 0, 0)
model_params = simulatedata.fit_sarima(dt['ret'][1:], order, seasonal_order)

In [ ]:
t = simulatedata.simulate_sarima(dt['ret'][1:], order, seasonal_order, model_params, len(dt), 10)
t

In [ ]:
simulated_price_data = simulatedata.construct_price_series(t, dt['close'][0], dt.index[0], freq)

In [ ]:
simulatedata.Evaluate_performance(dt['close'], simulated_price_data, lags=30)